# Define Table

In [43]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker


engine = create_engine('sqlite:///pmt.db', echo = True)
Session = sessionmaker(bind=engine)
sess = Session()

metadata = MetaData()
Base = declarative_base(metadata=metadata)

class PMT_USERS(Base):
    __tablename__ = 'pmt_users'
    email = Column(String, primary_key=True)
    username =  Column(String, nullable=False)
    password = Column(String, nullable=False)
    sec_groups = Column(String)
    
Base.metadata.create_all(engine)

2024-03-17 16:58:03,853 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2024-03-17 16:58:03,854 INFO sqlalchemy.engine.base.Engine ()
2024-03-17 16:58:03,855 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2024-03-17 16:58:03,856 INFO sqlalchemy.engine.base.Engine ()
2024-03-17 16:58:03,858 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("pmt_users")
2024-03-17 16:58:03,859 INFO sqlalchemy.engine.base.Engine ()
2024-03-17 16:58:03,861 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("pmt_users")
2024-03-17 16:58:03,862 INFO sqlalchemy.engine.base.Engine ()
2024-03-17 16:58:03,867 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE pmt_users (
	email VARCHAR NOT NULL, 
	username VARCHAR NOT NULL, 
	password VARCHAR NOT NULL, 
	sec_groups VARCHAR, 
	PRIMARY KEY (email)
)


2024-03-17 16:58:03,870 INFO sqlalchemy.engine.base.Engine ()
2024-03-17 16:58:03,877 INFO sqlalchemy.engine

In [66]:
user_1 = PMT_USERS(email='test@test.com', username='test', password="abc123", sec_groups='[]')

In [67]:
# perssist the user to the session - staging
sess.add(user_1)

In [68]:
pmt_user_1 = sess.query(PMT_USERS).filter_by(username='test').first()

2024-03-17 11:54:11,454 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2024-03-17 11:54:11,456 INFO sqlalchemy.engine.base.Engine INSERT INTO pmt_users (email, username, password, sec_groups) VALUES (?, ?, ?, ?)
2024-03-17 11:54:11,457 INFO sqlalchemy.engine.base.Engine ('test@test.com', 'test', 'abc123', '[]')
2024-03-17 11:54:11,463 INFO sqlalchemy.engine.base.Engine SELECT pmt_users.email AS pmt_users_email, pmt_users.username AS pmt_users_username, pmt_users.password AS pmt_users_password, pmt_users.sec_groups AS pmt_users_sec_groups 
FROM pmt_users 
WHERE pmt_users.username = ?
 LIMIT ? OFFSET ?
2024-03-17 11:54:11,466 INFO sqlalchemy.engine.base.Engine ('test', 1, 0)


In [70]:
pmt_user_1.username

'test'

In [71]:
# commit to db
sess.commit()

2024-03-17 11:55:11,665 INFO sqlalchemy.engine.base.Engine COMMIT


In [78]:
for email, username in sess.query(PMT_USERS.email, PMT_USERS.username):
    print(email, username)
    

2024-03-17 11:59:17,601 INFO sqlalchemy.engine.base.Engine SELECT pmt_users.email AS pmt_users_email, pmt_users.username AS pmt_users_username 
FROM pmt_users
2024-03-17 11:59:17,603 INFO sqlalchemy.engine.base.Engine ()
test@test.com test


In [75]:
users

In [76]:
for i in users:
    print(i)

2024-03-17 11:56:33,288 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2024-03-17 11:56:33,290 INFO sqlalchemy.engine.base.Engine SELECT pmt_users.email AS pmt_users_email, pmt_users.username AS pmt_users_username, pmt_users.password AS pmt_users_password, pmt_users.sec_groups AS pmt_users_sec_groups 
FROM pmt_users
2024-03-17 11:56:33,291 INFO sqlalchemy.engine.base.Engine ()


In [79]:
sess.close()

2024-03-17 12:00:57,971 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [88]:
sess.close()

# Map table

In [44]:
# For existing db
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker


engine = create_engine('sqlite:///pmt.db', echo=False)

# metadata = MetaData()
Base = declarative_base()

class PMT_USER(Base):
    __tablename__ = 'pmt_users'
    email = Column(String, primary_key=True)
    username =  Column(String)
    password = Column(String)
    sec_groups = Column(String)
    
# Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
sess = Session()

In [40]:
Column?

# Add users

In [93]:
# add 1 user
sess.add(PMT_USER(email="test2@test.com", username="test2", password="abc111", sec_groups='[]'))

In [96]:
sess.commit()

In [97]:
# add multiple users
sess.add_all(
    [
        PMT_USER(email="test3@test.com", username="test3", password="abc111", sec_groups='[]'),
        PMT_USER(email="test4@test.com", username="test4", password="abc111", sec_groups='[]')
    ]
)

# Get users

In [7]:
# all users
users = sess.query(PMT_USER)


In [8]:
# you can see it in stage
user2 = sess.query(PMT_USER).filter_by(username='test2').first()

# Update 

In [121]:
# set values for use using the setter
user2.password = 'aaa111'

In [122]:
user2.password

'aaa111'

In [123]:
sess.commit()

In [124]:
user2 = sess.query(PMT_USER).filter_by(username='test2').first()

In [125]:
user2.password

'aaa111'

In [126]:
user2.password

'aaa111'

In [2]:
user2

NameError: name 'user2' is not defined

# Functions

In [45]:
def get_all_users(sess):
    users = sess.query(PMT_USER)
    return [
        {
        "username": user.username, 
        "email": user.email, 
        "sec_groups": user.sec_groups
        } for user in users
    ] 

In [46]:
get_users(sess)

[]

In [47]:
import json

def get_user_sec_groups(sess, user_email):
    target_user = sess.query(PMT_USER).filter_by(email=user_email).first()
    return json.loads(target_user.sec_groups)

In [48]:
get_user_sec_groups(sess, 'test3@test.com')

AttributeError: 'NoneType' object has no attribute 'sec_groups'

In [61]:
def add_user(sess, user:dict):
    try:
        sess.add(PMT_USER(**user))
        sess.commit()
        return True
    except Exception as e:
        sess.rollback()
        raise e    

In [63]:
usr = {"username": "test1", "email":"test1@test.com", "sec_groups":'[]',"password":"abc123"} 
add_user(sess, usr)

True

In [59]:
sess.rollback()

In [39]:
sess.query(PMT_USER)[4].password